## Loading Important Libraries

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from PIL import Image
import matplotlib.image as immg
from pathlib import Path
import os
import gc
import cv2
from tqdm.notebook import tqdm
import zipfile
import io
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 150)

In [ ]:
home = Path('/kaggle/input/predict-volcanic-eruptions-ingv-oe')

In [ ]:
trn_dir = Path('../input/predict-volcanic-eruptions-ingv-oe/train')
tst_dir = Path('../input/predict-volcanic-eruptions-ingv-oe/test')

In [ ]:
df = pd.read_csv(home/'train.csv')

In [ ]:
df.segment_id = df.segment_id.apply(lambda x : str(x)+'.csv')

## Stratified Kfold for Regression

In [ ]:
from sklearn.model_selection import StratifiedKFold

def stra_kfold(data,target,targ_type,n_splits,seed=2020): 
    '''
    data = dataframe
    target = target variable
    targ_type = 'clas' for classfication | 'reg' for regression 
    n_splits = number of splits
    '''
    df = data.copy()
    df = df.sample(frac = 1., random_state = seed)
    
    if targ_type =='clas':
        df['kfold'] = -1
        y = df[target].values
        kf = StratifiedKFold(n_splits=n_splits,random_state = seed,shuffle = True)
        for fold ,(trn_,val_ )in enumerate(kf.split(X=df,y=y)):
            df.loc[val_,'kfold'] = fold
        return df
    elif targ_type =='reg':
        df['kfold'] = -1
        num_bins = int(np.floor(1 + np.log2(len(df))))
        df.loc[:,'bins'] = pd.cut(df[target],bins=num_bins,labels=False)
        kf = StratifiedKFold(n_splits = n_splits, random_state = seed, shuffle = True)
        for fold ,(trn_,val_ )in enumerate(kf.split(X=df,y=df.bins.values)):
            df.loc[val_,'kfold'] = fold
        return df
    else:
        return 0

In [ ]:
trn_df = stra_kfold(df,'time_to_eruption','reg',5)

In [ ]:
df.head(10)

In [ ]:
files = df.segment_id.tolist()

## Helpful functions to compress whole csv into single row features

In [ ]:
def make_df(a,name):
    return pd.DataFrame(data=np.array(a.values).reshape((1,10)),columns=[str(x)+"_"+str(name) for x in a.index])

In [ ]:
def add_feature(XX):
    X = XX.copy()
    nan = make_df(X.isna().sum(),'isna')
    X.fillna(value=-999,inplace=True)
    mean = make_df(X.mean(),'mean')
    std = make_df(X.std(),'std')
    var = make_df(X.var(),'var')
    maxi = make_df(X.max(),'max')
    lif = make_df(X.quantile(0.25)-(1.5*(X.quantile(0.75)-X.quantile(0.25))),'lif')
    uif = make_df(X.quantile(0.75)+(1.5*(X.quantile(0.75)-X.quantile(0.25))),'uif')
    mad = make_df(X.mad(),'mad')
    mini = make_df(X.min(),'min')
    quant5 = make_df(X.quantile(0.05),'quant5')
    quant10 = make_df(X.quantile(0.1),'quant10')
    quant15 = make_df(X.quantile(0.15),'quant15')
    quant25 = make_df(X.quantile(0.25),'quant25')
    quant35 = make_df(X.quantile(0.35),'quant35')
    quant50 = make_df(X.quantile(0.5),'quant50')
    quant65 = make_df(X.quantile(0.65),'quant65')
    quant75 = make_df(X.quantile(0.75),'quant75')
    quant85 = make_df(X.quantile(0.85),'quant85')
    quant90 = make_df(X.quantile(0.9),'quant90')
    quant95 = make_df(X.quantile(0.95),'quant95')
    iqr = make_df(X.quantile(0.75)-X.quantile(0.25),'iqr')
    skew = make_df(X.skew(),'skew')
    kurt = make_df(X.kurtosis(),'kurtosis')
    data = pd.concat([nan,mean,std,var,maxi,mini,quant5,quant10,quant15,quant25,quant35,quant50,quant65,
                      quant75,quant85,quant90,quant95,iqr,lif,uif,skew,kurt,mad],axis=1)
    return data

In [ ]:
from joblib import Parallel, delayed

## Will Be using joblib's parallel function to speed up data_processing

## Preparing train_data

In [ ]:
def make_all(fname):
    dfile = pd.read_csv(trn_dir/fname)
    new_row = add_feature(dfile)
    new_row['segment_id'] = fname
    return new_row

In [ ]:
out = Parallel(n_jobs=-1,verbose=1)(delayed(make_all)(i) for i in files)

In [ ]:
data_all = pd.concat(out,axis=0)

In [ ]:
data_all.head()

In [ ]:
data_all.to_csv('volcano_train_data.csv',index=False)

## Preparing Test Data

In [ ]:
sam = pd.read_csv('../input/predict-volcanic-eruptions-ingv-oe/sample_submission.csv')

In [ ]:
files_ts = [str(x)+'.csv' for x in sam.segment_id]

In [ ]:
def make_all_ts(fname):
    dfile = pd.read_csv(tst_dir/fname)
    new_row = add_feature(dfile)
    new_row['segment_id'] = fname
    return new_row

In [ ]:
out_ts = Parallel(n_jobs=-1,verbose=1)(delayed(make_all_ts)(i) for i in files_ts)

In [ ]:
data_test_all = pd.concat(out_ts,axis=0)

In [ ]:
data_test_all.to_csv('volcano_test_data.csv',index=False)

In [ ]:
data_trn = data_all.merge(trn_df,on='segment_id',how='left')

## Categorical features - is NaN
## Numeric features rest of them

In [ ]:
cat_names = data_trn.columns.tolist()[:10]
cont_names = data_trn.columns.tolist()[10:-4] 

## Training Using CatBoost Regressor

In [ ]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error

In [ ]:
test_sc = []
mae = []
for i in tqdm(range(5)):
    n = 3
    print('scaling_factor ',n)
    X_train,y_train = data_trn[data_trn.kfold!=i][cat_names+cont_names],data_trn[data_trn.kfold!=i]['time_to_eruption']**(1/n)
    X_test,y_test = data_trn[data_trn.kfold==i][cat_names+cont_names],data_trn[data_trn.kfold==i]['time_to_eruption']**(1/n)
    model = CatBoostRegressor(iterations = 2000,loss_function='RMSE',verbose=400)
    model.fit(X_train, y_train,cat_features=np.arange(10),eval_set=(X_test, y_test));
    y_true = y_test**n
    y_pred = model.predict(X_test)**n
    score = mean_absolute_error(y_true,y_pred)
    mae.append(score)
    print('--'*30)
    print('fold ' ,i,' mean_absolute_error : ',score)
    print('--'*30)
    ts_pred = model.predict(data_test_all[cat_names+cont_names])**n
    test_sc.append(ts_pred)

In [ ]:
print('avg_mean_absolute_error_5_fold : ',np.mean(mae))

In [ ]:
result = np.array(test_sc).mean(axis=0);result.shape

In [ ]:
l,m = df.time_to_eruption.quantile(.1),df.time_to_eruption.quantile(.90)

In [ ]:
sam['time_to_eruption'] = result
sam['time_to_eruption'].clip(l,m,inplace=True)

In [ ]:
sam.to_csv('submission_volcano2.csv',index=False)

In [ ]:
sam.head(10)